In [1]:
import tensorflow as tf 
from mtcnn.mtcnn import MTCNN
from PIL import Image
import numpy as np
import cv2

In [2]:
model = tf.keras.models.load_model("my_model3.h5")

In [3]:
def get_label(argument):
    labels = {0:"Angry" ,1:"Disgust",2:"Fear",3:"Happy",4:'Sad' , 5:'Surprise', 6:'Neutral'}
    return(labels.get(argument, "None"))


In [4]:
detector = MTCNN()
cap = cv2.VideoCapture(0)

if (cap.isOpened()== False):
    
    print("Error opening video stream")

while(cap.isOpened()):
    
    ret, frame = cap.read()
    if ret == True:
        
        #print(frame.shape) #480, 640, 3 
        frame = cv2.resize(frame,(800,600),interpolation = cv2.INTER_AREA)
        
        faces = detector.detect_faces(frame)
        images = [] 
        for face in faces:
            x, y, w, h = face['box']
            img = frame[y:y + h, x:x + w]
            img= cv2.resize(img, (48,48))
            images.append(img)
    
        labels = []
        for image in images:
            #gray = np.array(image)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            gray = gray/255
            gray = gray.reshape(-1,48, 48, 1)
            res = model.predict(gray)
            result_num = np.argmax(res)
            label = get_label(result_num)
            labels.append(label)
        for i in range(len(faces)):
            x, y, w, h = faces[i]['box']
            cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
            cv2.putText(frame, labels[i], (x+5, y-20),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        
        #precessed_frame = draw_pedection(frame)
        cv2.imshow('frame', frame)
        # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

        # Break the loop
    else:  
        break
# When everything done, release the video capture object
cap.release()

# Closes all the frames
cv2.destroyAllWindows()

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1268: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
